In [1]:
from pandas import pandas as pd
from __future__ import print_function
import sqlite3
import time
from apiclient import discovery
from httplib2 import Http
from oauth2client import file, client, tools

# built-in libraries
import datetime
import re
import sqlite3
import urllib
import csv

In [8]:
# read in state report on restaurant plan reviews as pd dataframe

plans = pd.read_csv("ftp://dbprftp.state.fl.us/pub/llweb/HR_plan_review.csv", encoding="ISO-8859-1")
plans.columns = ['region', 'county', 'biz_name', 'biz_street', 'biz_city',
          'biz_zip', 'biz_phone', 'biz_email', 'status', 'app_date',
          'review_date', 'type_app', 'type_rank', 'file_num', 'app_num',
          'lic_num', 'trans_var', 'unused', 'contact_name', 'contact_street',
          'contact_city', 'contact_zip', 'nation', 'contact_phone',
          'contact_email', 'alt_phone', 'alt_email']
#app_num = plans['app_num'].astype(str).astype(int) #convert app_num to int
plans['app_num'] = plans['app_num'].apply(int) # this does it also; better?

In [ ]:
# initial upload to db
conn = sqlite3.connect("planreview.sqlite")



In [ ]:
# filter to keep records that have app_num larger than those already posted,
# or an app_date after those already posted
conn = sqlite3.connect("planreview.sqlite")
df = pd.read_sql_query("select * from plans;", conn)
df
new_vals = plans[plans.app_num > app_num]

# or keep in pandas df and do the filtering there before export directly to db/sheet

In [ ]:


# take the latest records to add to db or send directly to Google sheet


# send values to Google sheet
SCOPES = 'https://www.googleapis.com/auth/spreadsheets'
store = file.Storage('storage.json')
creds = store.get()
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('client_secret_613750166644-e01b9e9p6ijs69ncs6u575td2ffugusm.apps.googleusercontent.com.json', SCOPES)
    creds = tools.run_flow(flow, store)
SHEETS = discovery.build('sheets', 'v4', http=creds.authorize(Http()))

data = {'properties': {'title': 'Restaurt plan reviews [%s]' % time.ctime()}}
res = SHEETS.spreadsheets().create(body=data).execute()
SHEET_ID = res['spreadsheetId']
print('Created "%s"' % res['properties']['title'])

FIELDS = ('rowid', 'region', 'county', 'biz_name', 'biz_street', 'biz_city',
          'biz_zip', 'biz_phone', 'biz_email', 'status', 'app_date',
          'review_date', 'type_app', 'type_rank', 'file_num', 'app_num',
          'lic_num', 'trans_var', 'unused', 'contact_name', 'contact_street',
          'contact_city', 'contact_zip', 'nation', 'contact_phone',
          'contact_email', 'alt_phone', 'alt_email', 'posted')
conn = sqlite3.connect('planreview.sqlite')
cur = conn.cursor()
rows = cur.execute('SELECT * FROM plans').fetchall()
conn.close()
rows.insert(0, FIELDS)
data = {'values': [row[:6] for row in rows]}

SHEETS.spreadsheets().values().update(spreadsheetId=SHEET_ID,
    range='A1', body=data, valueInputOption='RAW').execute()
print('Wrote data to Sheet:')
rows = SHEETS.spreadsheets().values().get(spreadsheetId=SHEET_ID,
    range='Sheet1').execute().get('values', [])
for row in rows:
    print(row)